# Behavior Cloning

### Data Augmentation and Formatting
Flip images and measurements

In [1]:
import csv
import cv2
import numpy as np

In [20]:
lines = []
with open("./data/driving_log.csv") as f:
    reader = csv.reader(f)
    for line in reader:
        filename = line[0].split('/')
        try:
            filename = '/'.join(['data',filename[-2],filename[-1]])
        except:
            print(filename, "::", line)
        lines.append([filename, *line[-4:]])

fw = open("./data/driving_log_updated.csv", "w")
for line in lines:
    fw.write("{},{},{},{},{}\n".format(*line))
    if abs(float(line[1])) > 0.3:
        new_filename = line[0][:-4]+"_flipped.jpg"
        img = cv2.imread(line[0], cv2.IMREAD_COLOR)
        img = np.fliplr(img)
        fw.write("{},{},{},{},{}\n".format(new_filename, -1.0 * float(line[1]), line[2], line[3], line[4]))
        cv2.imwrite(new_filename, img)
    
fw.close()

KeyboardInterrupt: 

Crop Images

In [3]:
# 55 off top
# 30 off bottom
#fw = open("./data/driving_log_updated_cropped.csv", "w")
#with open("./data/driving_log_updated.csv") as f:
#    reader = csv.reader(f)
#    for line in reader:
#        filename = line[0]
#        img = cv2.imread(filename)
#        img = img[55:-30, :]
#        f_split = filename.split("/")
#        f_split[-2] = "CROPPED_IMG"
#        new_filename = '/'.join(f_split)
#        fw.write("{},{},{},{},{}\n".format(new_filename, *line[1:]))
#        cv2.imwrite(new_filename, img)
#    
#fw.close()

### Define Datasets

In [4]:
import random
import csv

TEST_SPLIT_RATIO = 0.2
VALIDATION_SPLIT_RATIO = 0.1

train_data = []
test_data = []
validation_data = []

with open("./data/driving_log_updated.csv") as f:
    reader = csv.reader(f)
    for line in reader:
        rand = random.random()
        if rand < TEST_SPLIT_RATIO:
            test_data.append(line)
        elif rand < VALIDATION_SPLIT_RATIO+TEST_SPLIT_RATIO:
            validation_data.append(line)
        else:
            train_data.append(line)


In [5]:
total = 1.0* len(train_data)+len(test_data)+len(validation_data)
print(len(train_data), len(train_data)/total )
print(len(test_data), len(test_data)/total)
print(len(validation_data), len(validation_data)/total)


13322 0.7022667369530838
3787 0.1996309963099631
1861 0.09810226673695309


In [21]:
import sklearn
import random

def generator(samples, batch_size=32):
    num_samples = len(samples)
    samples = sklearn.utils.shuffle(samples)
    offset = 0
    while 1: # Loop forever so the generator never terminates
                    
        if offset >= len(samples):
            offset = 0
        vals = np.zeros((batch_size, 1))
        imgs = np.zeros((batch_size, 160, 320, 3))
        index = 0
        while index < batch_size:
            batch_sample = samples[offset]
            offset += 1
            if offset >= len(samples):
                offset = 0
            if None and abs(float(batch_sample[1])) < .4 and random.random() < .5:
                continue
            filename = batch_sample[0]
            center_image = cv2.imread(filename)
            imgs[index] = center_image
            try:
                float(batch_sample[1])
            except:
                print(batch_sample[1])
            vals[index][0] = float(batch_sample[1])
            index += 1

        # trim image to only see section with road
        X_train = imgs
        y_train = vals
            
        yield X_train, y_train


### Define Hyperparameters

In [35]:
epochs = 70
batch_size = 64

train_data_size = len(train_data)
test_data_size = len(test_data)
validation_data_size = len(validation_data)

### Define Model and Train

In [23]:
train_generator = generator(train_data, batch_size=batch_size)
validation_generator = generator(validation_data, batch_size=batch_size)
test_generator = generator(test_data, batch_size=batch_size)

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Lambda, Flatten, Dropout
from keras.layers.convolutional import Conv2D, Cropping2D
from keras.layers.advanced_activations import ELU
import keras.regularizers as regularizers

model = Sequential()
model.add(Cropping2D(cropping=((55,30),(0,0)), input_shape=(160, 320, 3)))
model.add(Lambda(lambda x: x/127.5 - 1.0, input_shape=(75, 320, 3)))
model.add(Conv2D(16, (5,5), strides=(4, 4)))
model.add(ELU())
model.add(Conv2D(32, (5,5), strides=(2, 2)))
model.add(ELU())
model.add(Conv2D(64, (3,3), strides=(2, 2)))
model.add(ELU())
model.add(Conv2D(64, (3,3), strides=(2, 2)))
model.add(ELU())
model.add(Flatten())
model.add(Dropout(.2))
model.add(Dense(1024,kernel_regularizer=regularizers.l2(0.01)))
model.add(ELU())
model.add(Dense(512,kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(.5))
model.add(ELU())
model.add(Dense(64,kernel_regularizer=regularizers.l2(0.01)))
model.add(ELU())
model.add(Dense(1))


Using TensorFlow backend.


In [10]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Lambda, Flatten, Dropout
from keras.layers.convolutional import Conv2D, Cropping2D
from keras.layers.advanced_activations import ELU

model = Sequential()
model.add(Cropping2D(cropping=((55,30),(0,0)), input_shape=(160, 320, 3)))
model.add(Lambda(lambda x: x/127.5 - 1.0, input_shape=(75, 320, 3)))
model.add(Conv2D(16, (8, 8), strides=(4, 4), padding="same"))
model.add(ELU())
model.add(Conv2D(32, (5, 5), strides=(2, 2), padding="same"))
model.add(ELU())
model.add(Conv2D(64, (5, 5), strides=(2, 2), padding="same"))
model.add(Flatten())
model.add(Dropout(.2))
model.add(ELU())
model.add(Dense(512))
model.add(Dropout(.5))
model.add(ELU())
model.add(Dense(1))

In [24]:
from keras.optimizers import Adam

model.compile(optimizer=Adam(lr=0.0001), loss='mse', metrics=[ 'mse'])

In [36]:
from keras.callbacks import ModelCheckpoint, Callback
checkpoint_callback = ModelCheckpoint('model{epoch:02d}.h5')
history = model.fit_generator(train_generator, steps_per_epoch=train_data_size/batch_size/3, \
                   validation_data=validation_generator, validation_steps=validation_data_size/batch_size, \
                   epochs=epochs, max_q_size=1, callbacks=[checkpoint_callback], initial_epoch=60)

Epoch 61/70
70/69 [==============================] - 14s - loss: 0.0215 - mean_squared_error: 0.0185 - val_loss: 0.0219 - val_mean_squared_error: 0.0187
Epoch 62/70
70/69 [==============================] - 13s - loss: 0.0215 - mean_squared_error: 0.0186 - val_loss: 0.0212 - val_mean_squared_error: 0.0184
Epoch 63/70
70/69 [==============================] - 13s - loss: 0.0215 - mean_squared_error: 0.0188 - val_loss: 0.0209 - val_mean_squared_error: 0.0181
Epoch 64/70
70/69 [==============================] - 13s - loss: 0.0210 - mean_squared_error: 0.0182 - val_loss: 0.0213 - val_mean_squared_error: 0.0184
Epoch 65/70
70/69 [==============================] - 13s - loss: 0.0212 - mean_squared_error: 0.0185 - val_loss: 0.0218 - val_mean_squared_error: 0.0189
Epoch 66/70
70/69 [==============================] - 13s - loss: 0.0213 - mean_squared_error: 0.0185 - val_loss: 0.0209 - val_mean_squared_error: 0.0182
Epoch 67/70
70/69 [==============================] - 13s - loss: 0.0209 - mean_squ

### Evaluate

In [37]:
metrics = model.evaluate_generator(test_generator, steps=test_data_size/batch_size)

In [33]:
for i in zip(model.metrics_names, metrics):
    print(i)

('loss', 0.020299196351940434)
('mean_squared_error', 0.017298458182873826)


In [38]:
for i in zip(model.metrics_names, metrics):
    print(i)

('loss', 0.019879928541680178)
('mean_squared_error', 0.017041624966077508)


In [39]:
model.save("model.h5")